**keyBERT** - Termextraktion mit BERT

Quellen: https://towardsdatascience.com/keyword-extraction-with-bert-724efca412ea und https://github.com/MaartenGr/KeyBERT

Wiederholung: BERT stellt Wörter als Vektoren dar. BERT sieht Wörter also nicht als feste Strings, sondern erkennt, dass Begriffe wie "See", "Teich" und "Gewässer" eine Ähnlichkeit aufweisen.

Wir können den folgenden Beispieltext verwenden.

In [ ]:
doc = """
         Supervised learning is the machine learning task of 
         learning a function that maps an input to an output based 
         on example input-output pairs.[1] It infers a function 
         from labeled training data consisting of a set of 
         training examples.[2] In supervised learning, each 
         example is a pair consisting of an input object 
         (typically a vector) and a desired output value (also 
         called the supervisory signal). A supervised learning 
         algorithm analyzes the training data and produces an 
         inferred function, which can be used for mapping new 
         examples. An optimal scenario will allow for the algorithm 
         to correctly determine the class labels for unseen 
         instances. This requires the learning algorithm to  
         generalize from the training data to unseen situations 
         in a 'reasonable' way (see inductive bias).
      """

**Preprocessing**

scikit-learn ist eine Sammlung von Funktionen für maschinelles Lernen mit Python. [1]

Wir können die Methode "CountVectorizer" aus scikit-learn verwenden. Sie überführt die Tokens eines Textdokuents in eine Matrix mit Wortzählungen. [2]

Der Wert "n_gram_range" kann angepasst werden, um die Länge der später extrahierten Keywords festzulegen.

Die Variable "candidates" enthält eine Liste von extrahierten "Termen" - sie heißt candidates, weil sie nur automatisch extrahierte Wörter enthält, für die manuell überprüft werden muss, ob sie tatsächlich Terme darstellen.

stop_words entfernt Stoppwörter (I, be, the, in, a, und so weiter)

[1] https://scikit-learn.org/stable

[2] https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

n_gram_range = (1, 1)
stop_words = "english"

# Extract candidate words/phrases
count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc])
candidates = count.get_feature_names()

Anschließend konvertieren wir die Wörter des Textdokuments mit BERT in Vektoren.

sentence-transformers ist eine Python-Bibliothek, mit der Wörter über ein vortrainiertes BERT-Modell in eine Vektor-Darstellung überführt werden können.

Im oberen Code-Feld wird die Bibliothek installiert.

Der untere Code zeigt, dass wir tatsächlich nicht BERT verwenden, sondern distilBERT [1]. Dabei handelt es sich um ein Modell, das ähnlich effektiv wie BERT ist, aber dabei einen wesentlich kleineres Aufbau hat, wodurch sich die Anwendung beschleunigt.

Das gesamte Dokument wird als ein (!) Vektor dargestellt ("doc embedding").
Die "Kandidaten" werden jeweils als Vektor dargestellt ("candidate embeddings").

[1] Sanh, V., Debut, L., Chaumond, J. & Wolf, T. (2019). Distilbert, a distilled version of BERT: smaller, faster, cheaper and lighter. In Proceedings of the NeurIPS EMC2 Workshop.

In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('distilbert-base-nli-mean-tokens')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

Wenn wir die "candidate embeddings" ausgeben, erhalten wir Zahlenwerte.

In [ ]:
for entry in candidate_embeddings:
    print (entry)

Jetzt können wir die einzelnen Wörter (die "Kandidaten") mit dem Gesamttext vergleichen.

Das ist schon alles. Mit top_n können wir ausgeben, wie viele Wörter im Ranking der Wörter, die dem Dokument am ähnlichsten sind, ausgegeben werden.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

for keyword in keywords:
    print (keyword)

Die Funktionen werden in der Klasse "keybert" bereitgestellt. [1]

Wir können sie hier viel einfacher verwenden.

Sie wird im folgenden Code importiert (!pip install keybert). Die Variable "doc" ist eine Wiederholung von oben - sie wird hier nur zur Übersichtlichkeit wiederholt.

[1] https://github.com/MaartenGr/KeyBERT

In [ ]:
!pip install keybert

from keybert import KeyBERT

doc = """
         Supervised learning is the machine learning task of learning a function that
         maps an input to an output based on example input-output pairs. It infers a
         function from labeled training data consisting of a set of training examples.
         In supervised learning, each example is a pair consisting of an input object
         (typically a vector) and a desired output value (also called the supervisory signal).
         A supervised learning algorithm analyzes the training data and produces an inferred function,
         which can be used for mapping new examples. An optimal scenario will allow for the
         algorithm to correctly determine the class labels for unseen instances. This requires
         the learning algorithm to generalize from the training data to unseen situations in a
         'reasonable' way (see inductive bias).
      """
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(doc)

for keyword in keywords:
    print (keyword)

Wir können im Textdokument (bisher: Die Variable "doc") die Wörter hervorheben, die als Keywords erkannt werden.

In [ ]:
keywords = kw_model.extract_keywords(doc, highlight=True)